In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
china = pd.read_csv("./data/china_city_riders.csv")
metro = pd.read_parquet("./data/metro_regions.parquet")

In [3]:
china.sample(2)

,station_1,station_2,s1_strip,s2_strip,trains,geometry_x,city1,geometry_y,city2,dist
3580,Hunchun Railway Station,Tumenbei Railway Station,hunchun,tumen,1,POINT Z (14515614.015607776 5296749.253939128 0),hunchun,POINT Z (14448459.512320956 5311115.004282691 0),tumen,68.673882
2054,Guangzhounan Railway Station,Jinanxi Railway Station,guangzhou,ji,14,POINT Z (12608493.456616975 2630984.2265344746 0),guangzhou,POINT Z (13011676.229528375 4393063.6699935505 0),ji,1807.617303


In [4]:
metro = metro[metro["country_code"] =="CN"]
metro.sample(2)

,metro,country_code,geometry,population,cities
111,Nanchang,CN,POINT (12896703.646043 3335482.390658),165182,"[Changleng, Jianguang, Gongqingcheng]"
48,Fuzhou,CN,POINT (13281095.413726 3006686.346673),1175223,"[Fuqing, Ningde, Jiangkou, Tantou, Fenggang, X..."


In [5]:
len(metro)

167

In [24]:
crossed = gpd.GeoDataFrame(metro.merge(
    metro,
    how="cross"
))
crossed["geometry_x"] = crossed["geometry_x"].apply(wkt.loads)
crossed["geometry_y"] = crossed["geometry_y"].apply(wkt.loads)
crossed = crossed.set_geometry("geometry_x")
crossed.crs = "EPSG:3857"
print(crossed.crs)
crossed["geometry_x"] = gpd.GeoSeries(crossed["geometry_x"])
crossed["geometry_y"] = gpd.GeoSeries(crossed["geometry_y"])
crossed["dist"] = crossed["geometry_x"].distance(crossed["geometry_y"]) / 1000
crossed = crossed[
    crossed["metro_x"] != crossed["metro_y"]
]

EPSG:3857


C:\Users\nicho\AppData\Local\Temp\ipykernel_8860\4264617719.py:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3857
Right CRS: None

  crossed["dist"] = crossed["geometry_x"].distance(crossed["geometry_y"]) / 1000


In [34]:
crossed["metro_x"] = crossed["metro_x"].str.lower()
crossed["metro_y"] = crossed["metro_y"].str.lower()
crossed.sample(2)

,metro_x,country_code_x,geometry_x,population_x,cities_x,metro_y,country_code_y,geometry_y,population_y,cities_y,dist
17751,xuanhua,CN,POINT (12808936.020 4955286.440),692602,[Zhangjiakou],huai'an,CN,POINT (13249153.399 3973695.262),193844,"[Gaogou, Zhongxing, Baoying]",1075.784635
12231,dazhou,CN,POINT (11962735.345 3660129.871),311133,"[Jukui, Liangping, Pingjin, Tieqiao, Zhonghe]",linyi,CN,POINT (13173858.009 4172454.052),509354,"[Jiehu, Shizilu, Xiazhuang, Bianzhuang, Sihu, ...",1315.026301


In [32]:
print(len(china))
china.sample(2)

9551


,station_1,station_2,s1_strip,s2_strip,trains,geometry_x,city1,geometry_y,city2,dist
721,Changchun Railway Station,Beijingnan Railway Station,changchun,beijing,125,POINT Z (13936456.989453055 5446180.397860233 0),changchun,POINT Z (12954622.78792938 4846115.115672885 0),beijing,1150.685336
6160,Shanghaihongqiao Railway Station,Rongcheng Railway Station,shanghaihongqiao,rongcheng,110,POINT Z (13504866.490043916 3658229.2535873577 0),shanghaihongqiao,POINT Z (13625743.883114694 4459081.466540753 0),rongcheng,809.923213


In [36]:
china = china.merge(
    crossed[["metro_x", "population_x", "metro_y", "population_y", "dist"]],
    how="inner",
    left_on=["city1", "city2"],
    right_on=["metro_x", "metro_y"]
)

In [39]:
china = china.drop(
    columns={
        "metro_x",
        "metro_y",
        "dist_y"
    }
).rename(
    columns={
        "population_x": "pop1",
        "population_y": "pop2"
    }
)

In [40]:
china.sample(2)

,station_1,station_2,s1_strip,s2_strip,trains,geometry_x,city1,geometry_y,city2,dist_x,pop1,pop2
939,Taiyuannan Railway Station,Hangzhoudong Railway Station,taiyuan,hangzhou,111,POINT Z (12532538.008450605 4559660.703318949 0),taiyuan,POINT Z (13381522.455762453 3541321.759809539 0),hangzhou,1325.816275,1462546,3888002
671,Ningbo Railway Station,Hefeinan Railway Station,ningbo,hefei,111,POINT Z (13528989.0397579 3486197.063300542 0),ningbo,POINT Z (13056105.745874763 3737160.9749078625 0),hefei,535.351748,3586581,138463


In [41]:
china.to_csv("./data/china_ridership_plus_pop.csv")

In [43]:
metro = pd.read_parquet("./data/metro_regions.parquet")
metro = metro[metro["country_code"] == "US"]
metro.sample(2)

,metro,country_code,geometry,population,cities
465,Rochester,US,POINT (-8640124.616835 5335560.693907),94273,"[Henrietta, Irondequoit]"
455,Louisville,US,POINT (-9546693.851932 4615403.917087),83692,"[Jeffersonville, New Albany]"


In [44]:
crossed = gpd.GeoDataFrame(metro.merge(metro, how="cross"))
crossed["geometry_x"] = crossed["geometry_x"].apply(wkt.loads)
crossed["geometry_y"] = crossed["geometry_y"].apply(wkt.loads)
crossed = crossed.set_geometry("geometry_x")
crossed.crs = "EPSG:3857"
print(crossed.crs)
crossed["geometry_x"] = gpd.GeoSeries(crossed["geometry_x"])
crossed["geometry_y"] = gpd.GeoSeries(crossed["geometry_y"])
crossed["dist"] = crossed["geometry_x"].distance(crossed["geometry_y"]) / 1000
crossed = crossed[crossed["metro_x"] != crossed["metro_y"]]

EPSG:3857


C:\Users\nicho\AppData\Local\Temp\ipykernel_8860\3283128325.py:9: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3857
Right CRS: None

  crossed["dist"] = crossed["geometry_x"].distance(crossed["geometry_y"]) / 1000


In [51]:
crossed = crossed.drop(columns=["country_code_x", "country_code_y"]).rename(
    columns={
        "metro_x": "city1",
        "metro_y": "city2",
        "population_x": "pop1",
        "population_y": "pop2",
    }
)

In [53]:
crossed.to_csv("./data/us_city_pairs.csv")